# Services (hour)

In [2]:
import pandas as pd

## Database connection

In [3]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [32]:
# Load dimensions
time_dimension = pd.read_sql_table("TIME_DIMENSION", OLAP_connection)
courier_dimension = pd.read_sql_table("COURIER_DIMENSION", OLAP_connection)
customer_dimension = pd.read_sql_table("CUSTOMER_DIMENSION", OLAP_connection)
office_dimension = pd.read_sql_table("OFFICE_DIMENSION", OLAP_connection)
service_status_dimension = pd.read_sql_table("SERVICE_STATUS_DIMENSION", OLAP_connection)

In [33]:
# Load fact table data from OLTP
services = pd.read_sql_table("mensajeria_servicio", OLTP_connection)
service_statuses = pd.read_sql_table("mensajeria_estadosservicio", OLTP_connection)

## Transformation

In [34]:
# Process service statuses to get latest status per service
# Convert 'hora' from datetime.time to string to avoid TypeError
service_statuses["hora_str"] = service_statuses["hora"].astype(str)

In [35]:
# Combine 'fecha' and 'hora_str' into a single datetime column
service_statuses["datetime"] = pd.to_datetime(
		service_statuses["fecha"]
) + pd.to_timedelta(service_statuses["hora_str"])

In [36]:
# Sort by 'datetime' and drop duplicates to get the latest status per 'servicio_id'
latest_status = service_statuses.sort_values("datetime").drop_duplicates(
		"servicio_id", keep="last"
)

In [37]:
# Rename columns for merging
latest_status = latest_status.rename(
		columns={
				"id": "status_record_id",
				"estado_id": "status_id",
				"servicio_id": "service_id",
		}
)

In [38]:
# Rename columns in services for consistency
services = services.rename(
		columns={
				"id": "service_id",
				"cliente_id": "customer_id",
				"mensajero_id": "courier_id",
				"origen_id": "origin_office_id",
				"destino_id": "destination_office_id",
		}
)

In [39]:
null_couriers = services["courier_id"].isna().sum()
print(f"Number of services with no assigned courier: {null_couriers}")

Number of services with no assigned courier: 727


In [40]:
# Ignore services with no assigned courier
services = services.dropna(subset=["courier_id"])

In [41]:
# Merge the latest status with services
services = services.merge(
		latest_status[["service_id", "status_id"]], on="service_id", how="left"
)

In [42]:
# Print number of services with no status
null_status = services["status_id"].isna().sum()
print(f"Number of services with no status: {null_status}")

Number of services with no status: 0


In [54]:
# Combine 'fecha_solicitud' and 'hora_solicitud' into 'request_time'
services["hora_solicitud_str"] = services["hora_solicitud"].astype(str)
services["request_time"] = pd.to_datetime(
		services["fecha_solicitud"]
) + pd.to_timedelta(services["hora_solicitud_str"])
services["request_time"] = services["request_time"].dt.floor("T")

In [57]:
# Create a mapping from datetime to time_id
# Ensure that 'time_dimension["date"]' is in datetime format
time_dimension["date"] = pd.to_datetime(time_dimension["date"])
time_mapping = dict(zip(time_dimension["date"], time_dimension["time_id"]))

In [58]:
# Map 'request_time' to 'time_id'
services["time_id"] = services["request_time"].map(time_mapping)

In [60]:
# Check for unmapped 'time_id's
unmapped_times = services["time_id"].isna().sum()
if unmapped_times > 0:
		print(f"Warning: {unmapped_times} records have unmapped 'request_time'.")

In [61]:
# Select necessary columns for the fact table
service_fact = services[
		[
				"service_id",
				"customer_id",
				"courier_id",
				"status_id",
				"origin_office_id",
				"destination_office_id",
				"time_id",
		]
]

In [62]:
service_fact.head(3)

,service_id,customer_id,courier_id,status_id,origin_office_id,destination_office_id,time_id
0,35,5,7.0,5,236,214,55398
1,46,4,12.0,5,262,238,75971
2,45,5,12.0,5,242,214,75954


In [ ]:
# Add service_fact_table_id
service_fact = service_fact.reset_index().rename(
		columns={"index": "service_fact_hour_table_id"}
)

In [41]:
service_fact.head(10)
# service_fact.shape

,service_fact_hour_table_id,service_id,customer_id,courier_id,status_id,origin_office_id,destination_office_id,time_id
0,0,35,5,7.0,5,236,214,55380
1,1,46,4,12.0,5,262,238,75960
2,2,45,5,12.0,5,242,214,75900
3,3,47,5,12.0,5,239,214,78420
4,4,31,5,7.0,2,257,230,37020
5,5,30,5,7.0,2,256,229,37020
6,6,29,5,7.0,6,256,229,37020
7,7,8,5,7.0,5,15,1,2400
8,8,22,5,7.0,1,15,1,8340
9,9,15,5,7.0,2,15,1,2460


In [42]:
null_couriers = service_fact["courier_id"].isna().sum()
print(f"Number of records with 'courier_id' empty: {null_couriers}")

Number of records with 'courier_id' empty: 0


## Load

In [43]:
# Save the fact table to OLAP
service_fact.to_sql(
		"SERVICE_FACT_HOUR_TABLE", OLAP_connection, if_exists="replace", index=False
)

703